In [1]:
import torch, matplotlib.pyplot as plt, math, gzip, pickle
from torch import tensor
from pathlib import Path
from fastcore.test import test_close

In [2]:
data_path = Path('data')/'mnist.pkl.gz'
with gzip.open(data_path, 'rb') as f:
    data = pickle.load(f, encoding='latin-1')
((x_train, y_train), (x_val, y_val), _) = data
(x_train, y_train, x_val, y_val) = map(tensor, (x_train, y_train, x_val, y_val))
x_train.shape, y_train.shape, x_val.shape, y_val.shape

(torch.Size([50000, 784]),
 torch.Size([50000]),
 torch.Size([10000, 784]),
 torch.Size([10000]))

In [3]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

In [4]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

In [5]:
def lin(x, w, b):
    return x@w+b

In [6]:
t = lin(x_val, w1, b1)
t.shape

torch.Size([10000, 50])

In [7]:
def relu(x):
    return x.clamp_min(0.)

In [8]:
t = relu(t)
t

tensor([[ 4.0894, 17.4166,  0.0000,  ...,  4.3990,  0.0000,  0.0000],
        [ 6.2843, 17.3042,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 5.6038,  6.4665,  0.0000,  ...,  5.0654,  0.0000,  0.0000],
        ...,
        [ 0.0000, 16.9027,  0.0000,  ...,  0.0000,  3.9315,  1.4503],
        [ 8.1895,  3.7898,  0.7553,  ...,  2.4189,  0.0000,  0.0000],
        [ 0.0000,  9.7781,  0.0000,  ...,  1.5611,  6.0280,  0.3595]])

In [9]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l1 = relu(l1)
    return lin(l1, w2, b2)

In [10]:
res = model(x_val)
res.shape

torch.Size([10000, 1])

In [11]:
(res-y_val).shape

torch.Size([10000, 10000])

In [12]:
(res[:,0]-y_val).shape

torch.Size([10000])

In [13]:
(res.squeeze()-y_val).shape

torch.Size([10000])

In [14]:
y_train,y_val = y_train.float(),y_val.float()
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [15]:
def mse(outp, targ):
    return (outp[:,0]-targ).pow(2).mean()

In [16]:
mse(preds, y_train)

tensor(1616.6124)

In [17]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [18]:
def lin_grad_v2(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0)

In [19]:
def forward_and_backward_v1(inp, targ):
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    out.g = 2*diff[:,None]/inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [20]:
def forward_and_backward_v2(inp, targ):
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    out.g = 2*diff[:,None]/inp.shape[0]
    lin_grad_v2(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad_v2(inp, l1, w1, b1)

In [21]:
%time forward_and_backward_v1(x_train, y_train)

CPU times: user 31.5 s, sys: 10min 21s, total: 10min 53s
Wall time: 4min 21s


In [22]:
%time forward_and_backward_v2(x_train, y_train)

CPU times: user 803 ms, sys: 368 ms, total: 1.17 s
Wall time: 260 ms


In [21]:
def get_grad(x):
    return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

In [22]:
def mkgrad(x):
    return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [23]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [24]:
loss = forward(xt2, y_train)
loss.backward()

In [25]:
for a, b in zip(grads, ptgrads):
    print(torch.allclose(a, b.grad, rtol=0.01))
    test_close(a, b.grad, eps=0.01)

True
True
True
True
True


In [40]:
class Relu:
    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp.clamp_min(0.)
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp > 0.).float()*self.out.g

In [41]:
class Lin:
    def __init__(self, w, b):
        self.w,self.b = w,b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = lin(self.inp, self.w, self.b)
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [42]:
class Mse:
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(self.inp, self.targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2.*(self.inp[:,0]-self.targ).unsqueeze(-1)/self.inp.shape[0]

In [43]:
class Model:
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [44]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

In [45]:
model = Model(w1, b1, w2, b2)

In [46]:
loss = model(x_train, y_train)
loss

tensor(1573.1453)

In [47]:
model.backward()